To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
# Skip restarting message in Colab
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm
!pip install --upgrade pillow
# If you are running this notebook on local, you need to install `diffusers` too
# !pip install diffusers
# Temporarily install a specific TRL nightly version
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [2]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 02-12 06:59:57 __init__.py:190] Automatically detected platform cuda.


Load up `Llama 3.1 8B Instruct`, and set parameters

In [3]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 512 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

Unsloth: Switching from Unsloth dynamic quant to normal quant since
we do not yet support fast inference for unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit
==((====))==  Unsloth 2025.2.5: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Unsloth: vLLM loading unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit with actual GPU utilization = 59.59%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 512. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.08 GB. Also swap space = 2 GB.
WARNING 02-12 07:00:10 config.py:2386] Casting torch.bfloat16 to torch.float16.
INFO 02-12 07:00:22 config.py:542] This model supports multiple tasks: {'score', 'generate', 'reward', 'classify', 'embed'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes config using kwargs = {'load_in_8bit': False, 'load_in_4bit': True, 'bnb_4bit_compute_dtype': 'float16', 'bnb_4bit_quant_storage': 'uint8', 'bnb_4bit_quant_type': 'nf4', 'bnb_4bit_use_double_quant': True, 'llm_int8_enable_fp32_cpu_offload': False, 'llm_int8_has_fp16_weight': False, 'llm_int8_skip_modules': ['lm_head', 'multi_modal_projector', 'merger', 'modality_projection'], 'llm_int8_threshold': 6

tokenizer_config.json:   0%|          | 0.00/6.77k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

INFO 02-12 07:01:35 model_runner.py:1562] Graph capturing finished in 38 secs, took 0.47 GiB
INFO 02-12 07:01:35 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 50.85 seconds


tokenizer_config.json:   0%|          | 0.00/6.77k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth 2025.2.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [5]:
var dataSetUnformatted = [
  {
    "conversations": [
      {"from": "human", "value": "Langfarm là gì?"},
      {"from": "gpt", "value": "Langfarm là một thương hiệu đặc sản nổi tiếng từ Đà Lạt, chuyên cung cấp các sản phẩm chất lượng cao như trái cây sấy, trà thảo mộc và các loại bánh mứt đặc sản."}
    ]
  },
  {
    "conversations": [
      {"from": "human", "value": "Địa chỉ trụ sở chính của Langfarm ở đâu?"},
      {"from": "gpt", "value": "Địa chỉ trụ sở chính của Langfarm là Số 1B, Hoàng Văn Thụ, Phường 5, Thành phố Đà Lạt, Tỉnh Lâm Đồng."}
    ]
  },
  {
    "conversations": [
      {"from": "human", "value": "Langfarm bán những sản phẩm gì?"},
      {"from": "gpt", "value": "Langfarm bán các sản phẩm như trái cây sấy dẻo, trái cây sấy giòn, trà atisô, trà tim sen, trà hoa cúc, bánh mứt đặc sản, và đông trùng hạ thảo."}
    ]
  },
  {
    "conversations": [
      {"from": "human", "value": "Langfarm Buffet là gì?"},
      {"from": "gpt", "value": "Langfarm Buffet là dịch vụ nơi khách hàng có thể thưởng thức hơn 50 loại đặc sản Đà Lạt với giá vé chỉ từ 69,000 đồng."}
    ]
  },
  {
    "conversations": [
      {"from": "human", "value": "Langfarm có bao nhiêu cửa hàng?"},
      {"from": "gpt", "value": "Langfarm hiện có hơn 35 cửa hàng tại các thành phố lớn như Đà Lạt, Hồ Chí Minh, Hà Nội, Nha Trang, Đà Nẵng, và Hải Phòng."}
    ]
  },
  {
    "conversations": [
      {"from": "human", "value": "Làm thế nào để liên hệ với Langfarm?"},
      {"from": "gpt", "value": "Bạn có thể liên hệ với Langfarm qua email hotro@langfarm.com hoặc hotline 19009065."}
    ]
  },
  {
    "conversations": [
      {"from": "human", "value": "Tại sao Langfarm được yêu thích?"},
      {"from": "gpt", "value": "Langfarm được yêu thích vì mang đến các sản phẩm đặc sản an toàn, chất lượng cao, và thiết kế đẹp mắt, đáp ứng nhu cầu của cả khách hàng trong nước và quốc tế."}
    ]
  },
  {
    "conversations": [
      {"from": "human", "value": "Langfarm Store khác gì Langfarm Buffet?"},
      {"from": "gpt", "value": "Langfarm Store là chuỗi cửa hàng chuyên bán các sản phẩm đặc sản, trong khi Langfarm Buffet là nơi khách hàng có thể thưởng thức hơn 50 loại đặc sản với một mức giá cố định."}
    ]
  },
  {
    "conversations": [
      {"from": "human", "value": "Trà thảo mộc của Langfarm có gì đặc biệt?"},
      {"from": "gpt", "value": "Trà thảo mộc của Langfarm được làm từ nguyên liệu tự nhiên, mang lại hương vị thơm ngon và có lợi cho sức khỏe, như trà atisô giúp mát gan và trà tim sen giúp dễ ngủ."}
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bánh biscotti nông sản Đà Lạt đặc sản Langfarm Hộp raisu 260g"
      },
      {
        "from": "gpt",
        "value": "109000 đồng, Bánh to gấp đôi, hạt nhiều gấp ba, 1 bánh cho 1 bữa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bánh cookie macaron Đà Lạt đặc sản Langfarm Hũ tobita 80g"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Làm từ bột hạnh nhân cao cấp, hương vị nông sản, giòn tan trong miệng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bánh flan yến sấy thăng hoa đặc sản Yumsea x Langfarm Hộp pillow 2 bánh"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Tổ yến nguyên chất, bánh ngon, bổ dưỡng, tan trong miệng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bắp rang tỏi ớt đặc sản Yumsea x Langfarm Bịch washi 150g"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Giòn ngon bắt miệng, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bắp rang tỏi ớt đặc sản Yumsea x Langfarm Bịch washi 150g Hũ tobita 150g"
      },
      {
        "from": "gpt",
        "value": "61000 đồng, Giòn ngon bắt miệng, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bắp rang tỏi ớt đặc sản Yumsea x Langfarm Hũ tobita 150g Bịch washi 150g"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Giòn ngon bắt miệng, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bắp rang tỏi ớt đặc sản Yumsea x Langfarm Hũ tobita 150g"
      },
      {
        "from": "gpt",
        "value": "61000 đồng, Giòn ngon bắt miệng, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bộ bao bì hộp quà tự chọn Langfarm Size 310x250x120"
      },
      {
        "from": "gpt",
        "value": "109000 đồng, Vân gỗ ép thủy lực, hoàn thiện tỉ mỉ, bao gồm túi giấy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bộ bao bì hộp quà tự chọn Langfarm Size 310x250x120 Size 270x165x120"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Vân gỗ ép thủy lực, hoàn thiện tỉ mỉ, bao gồm túi giấy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bộ bao bì hộp quà tự chọn Langfarm Size 270x165x120 Size 310x250x120"
      },
      {
        "from": "gpt",
        "value": "109000 đồng, Vân gỗ ép thủy lực, hoàn thiện tỉ mỉ, bao gồm túi giấy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bộ bao bì hộp quà tự chọn Langfarm Size 270x165x120"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Vân gỗ ép thủy lực, hoàn thiện tỉ mỉ, bao gồm túi giấy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Hũ tobita 110g"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Hũ tobita 110g Bịch kraft 150g"
      },
      {
        "from": "gpt",
        "value": "162000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Hũ tobita 110g Bịch mawashi 450g"
      },
      {
        "from": "gpt",
        "value": "467000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Hũ tobita 110g Hũ tobita 150g"
      },
      {
        "from": "gpt",
        "value": "173000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Bịch kraft 150g Hũ tobita 110g"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Bịch kraft 150g"
      },
      {
        "from": "gpt",
        "value": "162000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Bịch kraft 150g Bịch mawashi 450g"
      },
      {
        "from": "gpt",
        "value": "467000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Bịch kraft 150g Hũ tobita 150g"
      },
      {
        "from": "gpt",
        "value": "173000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Bịch mawashi 450g Hũ tobita 110g"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Bịch mawashi 450g Bịch kraft 150g"
      },
      {
        "from": "gpt",
        "value": "162000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Bịch mawashi 450g"
      },
      {
        "from": "gpt",
        "value": "467000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Bịch mawashi 450g Hũ tobita 150g"
      },
      {
        "from": "gpt",
        "value": "173000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Hũ tobita 150g Hũ tobita 110g"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Hũ tobita 150g Bịch kraft 150g"
      },
      {
        "from": "gpt",
        "value": "162000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Hũ tobita 150g Bịch mawashi 450g"
      },
      {
        "from": "gpt",
        "value": "467000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bò khô xé sợi đặc sản Yumsea x Langfarm Hũ tobita 150g"
      },
      {
        "from": "gpt",
        "value": "173000 đồng, Bò thật 100%, tẩm ướp đậm đà, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bông atisô sấy khô đặc sản Langfarm Hũ yama 100g"
      },
      {
        "from": "gpt",
        "value": "153000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bông atisô sấy khô đặc sản Langfarm Hũ yama 100g Bịch 225g"
      },
      {
        "from": "gpt",
        "value": "274000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bông atisô sấy khô đặc sản Langfarm Bịch 225g Hũ yama 100g"
      },
      {
        "from": "gpt",
        "value": "153000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bông atisô sấy khô đặc sản Langfarm Bịch 225g"
      },
      {
        "from": "gpt",
        "value": "274000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột bồ công anh đặc sản Bột Lá x Langfarm Bịch hanko 100g"
      },
      {
        "from": "gpt",
        "value": "153000 đồng, Từ nông trại Đà Lạt, bột nguyên chất, công nghệ sấy lạnh, xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột cải xoăn kale đặc sản Bột Lá x Langfarm Bịch hanko 100g"
      },
      {
        "from": "gpt",
        "value": "175000 đồng, Từ nông trại Đà Lạt, bột nguyên chất, công nghệ sấy lạnh, xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột cần tây đặc sản Bột Lá x Langfarm Bịch hanko 100g"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Từ nông trại Đà Lạt, bột nguyên chất, công nghệ sấy lạnh, xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột tía tô đặc sản Bột Lá x Langfarm Bịch hanko 100g"
      },
      {
        "from": "gpt",
        "value": "109000 đồng, Từ nông trại Đà Lạt, bột nguyên chất, công nghệ sấy lạnh, xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột trà xanh matcha sữa Nhật Bản Matchi Matcha x Langfarm Hộp hanko túi 16 túi 12g Matcha sữa"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Lá trà nhập khẩu Nhật Bản, bột nguyên chất, công nghệ xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột trà xanh Nhật Bản Matchi Matcha x Langfarm Hộp hanko 16 túi 4g Matcha nguyên chất"
      },
      {
        "from": "gpt",
        "value": "79000 đồng, Lá trà nhập khẩu Nhật Bản, bột nguyên chất, công nghệ xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột trà xanh Nhật Bản Matchi Matcha x Langfarm Hộp hanko 16 túi 4g Hộp 1000g Matcha nguyên chất"
      },
      {
        "from": "gpt",
        "value": "910000 đồng, Lá trà nhập khẩu Nhật Bản, bột nguyên chất, công nghệ xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột trà xanh Nhật Bản Matchi Matcha x Langfarm Hộp hanko 16 túi 4g Bịch hanko 100g Matcha nguyên chất"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Lá trà nhập khẩu Nhật Bản, bột nguyên chất, công nghệ xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột trà xanh Nhật Bản Matchi Matcha x Langfarm Hộp 1000g Hộp hanko 16 túi 4g Matcha nguyên chất"
      },
      {
        "from": "gpt",
        "value": "79000 đồng, Lá trà nhập khẩu Nhật Bản, bột nguyên chất, công nghệ xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột trà xanh Nhật Bản Matchi Matcha x Langfarm Hộp 1000g Matcha nguyên chất"
      },
      {
        "from": "gpt",
        "value": "910000 đồng, Lá trà nhập khẩu Nhật Bản, bột nguyên chất, công nghệ xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột trà xanh Nhật Bản Matchi Matcha x Langfarm Hộp 1000g Bịch hanko 100g Matcha nguyên chất"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Lá trà nhập khẩu Nhật Bản, bột nguyên chất, công nghệ xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột trà xanh Nhật Bản Matchi Matcha x Langfarm Bịch hanko 100g Hộp hanko 16 túi 4g Matcha nguyên chất"
      },
      {
        "from": "gpt",
        "value": "79000 đồng, Lá trà nhập khẩu Nhật Bản, bột nguyên chất, công nghệ xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột trà xanh Nhật Bản Matchi Matcha x Langfarm Bịch hanko 100g Hộp 1000g Matcha nguyên chất"
      },
      {
        "from": "gpt",
        "value": "910000 đồng, Lá trà nhập khẩu Nhật Bản, bột nguyên chất, công nghệ xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bột trà xanh Nhật Bản Matchi Matcha x Langfarm Bịch hanko 100g Matcha nguyên chất"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Lá trà nhập khẩu Nhật Bản, bột nguyên chất, công nghệ xay chậm siêu mịn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bưởi sấy dẻo đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bưởi sấy dẻo đặc sản Langfarm Hũ tobita 200g Bịch mawashi 650g"
      },
      {
        "from": "gpt",
        "value": "197000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bưởi sấy dẻo đặc sản Langfarm Hũ tobita 200g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bưởi sấy dẻo đặc sản Langfarm Bịch mawashi 650g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bưởi sấy dẻo đặc sản Langfarm Bịch mawashi 650g"
      },
      {
        "from": "gpt",
        "value": "197000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bưởi sấy dẻo đặc sản Langfarm Bịch mawashi 650g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bưởi sấy dẻo đặc sản Langfarm Bịch kraft 225g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bưởi sấy dẻo đặc sản Langfarm Bịch kraft 225g Bịch mawashi 650g"
      },
      {
        "from": "gpt",
        "value": "197000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Bưởi sấy dẻo đặc sản Langfarm Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cá cơm kho ăn liền đặc sản Yumsea x Langfarm Hũ tobita 170g"
      },
      {
        "from": "gpt",
        "value": "132000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cá cơm kho ăn liền đặc sản Yumsea x Langfarm Hũ tobita 170g Bịch kraft 150g"
      },
      {
        "from": "gpt",
        "value": "100000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cá cơm kho ăn liền đặc sản Yumsea x Langfarm Bịch kraft 150g Hũ tobita 170g"
      },
      {
        "from": "gpt",
        "value": "132000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cá cơm kho ăn liền đặc sản Yumsea x Langfarm Bịch kraft 150g"
      },
      {
        "from": "gpt",
        "value": "100000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cá cơm sấy giòn ăn liền đặc sản Yumsea x Langfarm Bịch kraft 135g"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cá cơm sấy giòn ăn liền đặc sản Yumsea x Langfarm Bịch kraft 135g Hũ tobita 120g"
      },
      {
        "from": "gpt",
        "value": "90000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cá cơm sấy giòn ăn liền đặc sản Yumsea x Langfarm Hũ tobita 120g Bịch kraft 135g"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cá cơm sấy giòn ăn liền đặc sản Yumsea x Langfarm Hũ tobita 120g"
      },
      {
        "from": "gpt",
        "value": "90000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cam sấy dẻo đặc sản Langfarm Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "105000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cam sấy dẻo đặc sản Langfarm Bịch kraft 225g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cam sấy dẻo đặc sản Langfarm Hũ tobita 200g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "105000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cam sấy dẻo đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cao atisô thảo mộc Langfarm Hộp antique 150g"
      },
      {
        "from": "gpt",
        "value": "164000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cao atisô thảo mộc Langfarm Hộp antique 150g 3 hũ 150g"
      },
      {
        "from": "gpt",
        "value": "417000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cao atisô thảo mộc Langfarm Hộp antique 150g Hũ 150g"
      },
      {
        "from": "gpt",
        "value": "142000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cao atisô thảo mộc Langfarm 3 hũ 150g Hộp antique 150g"
      },
      {
        "from": "gpt",
        "value": "164000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cao atisô thảo mộc Langfarm 3 hũ 150g"
      },
      {
        "from": "gpt",
        "value": "417000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cao atisô thảo mộc Langfarm 3 hũ 150g"
      },
      {
        "from": "gpt",
        "value": "142000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cao atisô thảo mộc Langfarm Hũ 150g Hộp antique 150g"
      },
      {
        "from": "gpt",
        "value": "164000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cao atisô thảo mộc Langfarm Hũ 150g 3 hũ 150g"
      },
      {
        "from": "gpt",
        "value": "417000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cao atisô thảo mộc Langfarm Hũ 150g"
      },
      {
        "from": "gpt",
        "value": "142000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cao ống atisô thảo mộc Langfarm Hộp antique 10 ống"
      },
      {
        "from": "gpt",
        "value": "65000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da (Đậm đặc gấp đôi)"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê bột đặc sản Langfarm Bịch 250g Phối trộn vừa"
      },
      {
        "from": "gpt",
        "value": "131000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê bột đặc sản Langfarm Bịch 250g Phối trộn nhẹ"
      },
      {
        "from": "gpt",
        "value": "142000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê bột đặc sản Langfarm Bịch 250g Phối trộn đậm"
      },
      {
        "from": "gpt",
        "value": "127000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê bột đặc sản Langfarm Bịch 250g Phối trộn vừa"
      },
      {
        "from": "gpt",
        "value": "131000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê bột đặc sản Langfarm Bịch 250g Phối trộn nhẹ"
      },
      {
        "from": "gpt",
        "value": "142000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê bột đặc sản Langfarm Bịch 250g Phối trộn đậm"
      },
      {
        "from": "gpt",
        "value": "127000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê bột đặc sản Langfarm Bịch 250g Phối trộn vừa"
      },
      {
        "from": "gpt",
        "value": "131000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê bột đặc sản Langfarm Bịch 250g Phối trộn nhẹ"
      },
      {
        "from": "gpt",
        "value": "142000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê bột đặc sản Langfarm Bịch 250g Phối trộn đậm"
      },
      {
        "from": "gpt",
        "value": "127000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê nguyên hạt đặc sản Langfarm Bịch eco 200g Phối trộn đậm"
      },
      {
        "from": "gpt",
        "value": "102000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê nguyên hạt đặc sản Langfarm Bịch eco 200g Phối trộn vừa"
      },
      {
        "from": "gpt",
        "value": "106000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê nguyên hạt đặc sản Langfarm Bịch eco 200g Phối trộn nhẹ"
      },
      {
        "from": "gpt",
        "value": "115000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê nguyên hạt đặc sản Langfarm Bịch eco 200g Phối trộn đậm"
      },
      {
        "from": "gpt",
        "value": "102000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê nguyên hạt đặc sản Langfarm Bịch eco 200g Phối trộn vừa"
      },
      {
        "from": "gpt",
        "value": "106000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê nguyên hạt đặc sản Langfarm Bịch eco 200g Phối trộn nhẹ"
      },
      {
        "from": "gpt",
        "value": "115000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê nguyên hạt đặc sản Langfarm Bịch eco 200g Phối trộn đậm"
      },
      {
        "from": "gpt",
        "value": "102000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê nguyên hạt đặc sản Langfarm Bịch eco 200g Phối trộn vừa"
      },
      {
        "from": "gpt",
        "value": "106000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cà phê nguyên hạt đặc sản Langfarm Bịch eco 200g Phối trộn nhẹ"
      },
      {
        "from": "gpt",
        "value": "115000 đồng, Từ cao nguyên Lâm Đồng, công nghệ rang hot air từ IMF"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chanh dây sấy dẻo đặc sản Langfarm Hũ tobita 225g"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chanh dây sấy dẻo đặc sản Langfarm Hũ tobita 225g Bịch kraft 250g"
      },
      {
        "from": "gpt",
        "value": "92000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chanh dây sấy dẻo đặc sản Langfarm Bịch kraft 250g Hũ tobita 225g"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chanh dây sấy dẻo đặc sản Langfarm Bịch kraft 250g"
      },
      {
        "from": "gpt",
        "value": "92000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối la ba Đà Lạt sấy dẻo đặc sản Langfarm Bịch chân không 300g"
      },
      {
        "from": "gpt",
        "value": "65000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối la ba Đà Lạt sấy dẻo đặc sản Langfarm Bịch chân không 300g Bịch tray 300g"
      },
      {
        "from": "gpt",
        "value": "65000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối la ba Đà Lạt sấy dẻo đặc sản Langfarm Bịch tray 300g Bịch chân không 300g"
      },
      {
        "from": "gpt",
        "value": "65000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối la ba Đà Lạt sấy dẻo đặc sản Langfarm Bịch tray 300g"
      },
      {
        "from": "gpt",
        "value": "65000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối sấy dẻo đặc sản Langfarm Bịch chân không 300g"
      },
      {
        "from": "gpt",
        "value": "73000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối sấy giòn đặc sản Langfarm Hũ tobita 100g"
      },
      {
        "from": "gpt",
        "value": "30000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối sấy giòn đặc sản Langfarm Hũ tobita 100g Bịch mawashi 400g"
      },
      {
        "from": "gpt",
        "value": "73000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối sấy giòn đặc sản Langfarm Hũ tobita 100g Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "29000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối sấy giòn đặc sản Langfarm Bịch mawashi 400g Hũ tobita 100g"
      },
      {
        "from": "gpt",
        "value": "30000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối sấy giòn đặc sản Langfarm Bịch mawashi 400g"
      },
      {
        "from": "gpt",
        "value": "73000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối sấy giòn đặc sản Langfarm Bịch mawashi 400g Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "29000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối sấy giòn đặc sản Langfarm Bịch kraft 120g Hũ tobita 100g"
      },
      {
        "from": "gpt",
        "value": "30000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối sấy giòn đặc sản Langfarm Bịch kraft 120g Bịch mawashi 400g"
      },
      {
        "from": "gpt",
        "value": "73000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Chuối sấy giòn đặc sản Langfarm Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "29000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kawamura 2 combo"
      },
      {
        "from": "gpt",
        "value": "166000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kawamura 2 combo Kintsugi 4 combo"
      },
      {
        "from": "gpt",
        "value": "371000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kawamura 2 combo Kintsugi 1 combo"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kawamura 2 combo Kawamura 5 combo"
      },
      {
        "from": "gpt",
        "value": "355000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kintsugi 4 combo Kawamura 2 combo"
      },
      {
        "from": "gpt",
        "value": "166000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kintsugi 4 combo"
      },
      {
        "from": "gpt",
        "value": "371000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kintsugi 4 combo Kintsugi 1 combo"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kintsugi 4 combo Kawamura 5 combo"
      },
      {
        "from": "gpt",
        "value": "355000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kintsugi 1 combo Kawamura 2 combo"
      },
      {
        "from": "gpt",
        "value": "166000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kintsugi 1 combo Kintsugi 4 combo"
      },
      {
        "from": "gpt",
        "value": "371000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kintsugi 1 combo"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kintsugi 1 combo Kawamura 5 combo"
      },
      {
        "from": "gpt",
        "value": "355000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kawamura 5 combo Kawamura 2 combo"
      },
      {
        "from": "gpt",
        "value": "166000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kawamura 5 combo Kintsugi 4 combo"
      },
      {
        "from": "gpt",
        "value": "371000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kawamura 5 combo Kintsugi 1 combo"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Combo yến baby thiên nhiên Yumsea x Langfarm Kawamura 5 combo"
      },
      {
        "from": "gpt",
        "value": "355000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cỏ ngọt sấy khô đặc sản Langfarm Hũ yama 70g"
      },
      {
        "from": "gpt",
        "value": "54000 đồng, Giúp tạo vị ngọt tự nhiên, hỗ trợ kiểm soát đường huyết và tốt cho người ăn kiêng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cỏ ngọt sấy khô đặc sản Langfarm Hũ yama 70g Bịch 225g"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Giúp tạo vị ngọt tự nhiên, hỗ trợ kiểm soát đường huyết và tốt cho người ăn kiêng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cỏ ngọt sấy khô đặc sản Langfarm Bịch 225g Hũ yama 70g"
      },
      {
        "from": "gpt",
        "value": "54000 đồng, Giúp tạo vị ngọt tự nhiên, hỗ trợ kiểm soát đường huyết và tốt cho người ăn kiêng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Cỏ ngọt sấy khô đặc sản Langfarm Bịch 225g"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Giúp tạo vị ngọt tự nhiên, hỗ trợ kiểm soát đường huyết và tốt cho người ăn kiêng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu hoà lan muối đặc sản Langfarm Hũ tobita 180g"
      },
      {
        "from": "gpt",
        "value": "38000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu hoà lan muối đặc sản Langfarm Hũ tobita 180g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "39000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu hoà lan muối đặc sản Langfarm Bịch kraft 225g Hũ tobita 180g"
      },
      {
        "from": "gpt",
        "value": "38000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu hoà lan muối đặc sản Langfarm Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "39000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu hoà lan wasabi đặc sản Langfarm Hũ tobita 150g"
      },
      {
        "from": "gpt",
        "value": "42000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu hoà lan wasabi đặc sản Langfarm Hũ tobita 150g Bịch kraft 200g"
      },
      {
        "from": "gpt",
        "value": "47000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu hoà lan wasabi đặc sản Langfarm Bịch kraft 200g Hũ tobita 150g"
      },
      {
        "from": "gpt",
        "value": "42000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu hoà lan wasabi đặc sản Langfarm Bịch kraft 200g"
      },
      {
        "from": "gpt",
        "value": "47000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng nước cốt dừa đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "41000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng nước cốt dừa đặc sản Langfarm Hũ tobita 200g Bịch kraft 250g"
      },
      {
        "from": "gpt",
        "value": "43000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng nước cốt dừa đặc sản Langfarm Bịch kraft 250g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "41000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng nước cốt dừa đặc sản Langfarm Bịch kraft 250g"
      },
      {
        "from": "gpt",
        "value": "43000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng sấy rau củ đặc sản Langfarm Hũ tobita 100g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng sấy rau củ đặc sản Langfarm Hũ tobita 100g Bịch kraft 135g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng sấy rau củ đặc sản Langfarm Bịch kraft 135g Hũ tobita 100g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng sấy rau củ đặc sản Langfarm Bịch kraft 135g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng sấy sữa bắp đặc sản Langfarm Hũ tobita 100g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng sấy sữa bắp đặc sản Langfarm Hũ tobita 100g Bịch kraft 125g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng sấy sữa bắp đặc sản Langfarm Bịch kraft 125g Hũ tobita 100g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đậu phộng sấy sữa bắp đặc sản Langfarm Bịch kraft 125g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đông trùng hạ thảo sấy khô Langfarm Hộp 10g"
      },
      {
        "from": "gpt",
        "value": "399000 đồng, Giúp tăng cường sức đề kháng, bồi bổ sức khỏe và hỗ trợ cải thiện chức năng hô hấp"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đu đủ sấy dẻo đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "75000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đu đủ sấy dẻo đặc sản Langfarm Hũ tobita 200g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đu đủ sấy dẻo đặc sản Langfarm Bịch kraft 225g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "75000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Đu đủ sấy dẻo đặc sản Langfarm Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch mawashi 750g Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Hũ tobita 200g Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "229000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Trái cây"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "318000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "261000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Bịch mawashi 750g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "292000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Trái cây"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "76000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Hũ tobita 200g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Trái cây"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g 45% yến mạch"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g 15% yến mạch"
      },
      {
        "from": "gpt",
        "value": "133000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Siêu hạt"
      },
      {
        "from": "gpt",
        "value": "112000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Granola đặc sản Langfarm Bịch kraft 300g Siêu hạt đặc sản"
      },
      {
        "from": "gpt",
        "value": "123000 đồng, Hạt chất lượng, trái cây tuyển lựa, nướng phủ mật ong, healthy, eat clean"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạnh nhân rang muối biển đặc sản Langfarm Hũ tobita 240g"
      },
      {
        "from": "gpt",
        "value": "137000 đồng, Hạnh nhân nhập khẩu loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạnh nhân rang muối biển đặc sản Langfarm Hũ tobita 240g Bịch kraft 275g"
      },
      {
        "from": "gpt",
        "value": "150000 đồng, Hạnh nhân nhập khẩu loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạnh nhân rang muối biển đặc sản Langfarm Bịch kraft 275g Hũ tobita 240g"
      },
      {
        "from": "gpt",
        "value": "137000 đồng, Hạnh nhân nhập khẩu loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạnh nhân rang muối biển đặc sản Langfarm Bịch kraft 275g"
      },
      {
        "from": "gpt",
        "value": "150000 đồng, Hạnh nhân nhập khẩu loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vàng đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "109000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vàng đặc sản Langfarm Hũ tobita 200g Bịch mawashi 750g"
      },
      {
        "from": "gpt",
        "value": "369000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vàng đặc sản Langfarm Hũ tobita 200g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "120000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vàng đặc sản Langfarm Bịch mawashi 750g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "109000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vàng đặc sản Langfarm Bịch mawashi 750g"
      },
      {
        "from": "gpt",
        "value": "369000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vàng đặc sản Langfarm Bịch mawashi 750g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "120000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vàng đặc sản Langfarm Bịch kraft 225g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "109000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vàng đặc sản Langfarm Bịch kraft 225g Bịch mawashi 750g"
      },
      {
        "from": "gpt",
        "value": "369000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vàng đặc sản Langfarm Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "120000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vỏ lụa đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vỏ lụa đặc sản Langfarm Hũ tobita 200g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "120000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vỏ lụa đặc sản Langfarm Bịch kraft 225g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt điều vỏ lụa đặc sản Langfarm Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "120000 đồng, Hạt điều Bình Phước loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hà thủ ô sấy khô đặc sản Langfarm Hũ yama 200g"
      },
      {
        "from": "gpt",
        "value": "105000 đồng, Giúp bổ thận, dưỡng huyết, và hỗ trợ làm đen tóc, chống lão hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hà thủ ô sấy khô đặc sản Langfarm Hũ yama 200g Bịch 450g"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Giúp bổ thận, dưỡng huyết, và hỗ trợ làm đen tóc, chống lão hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hà thủ ô sấy khô đặc sản Langfarm Bịch 450g Hũ yama 200g"
      },
      {
        "from": "gpt",
        "value": "105000 đồng, Giúp bổ thận, dưỡng huyết, và hỗ trợ làm đen tóc, chống lão hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hà thủ ô sấy khô đặc sản Langfarm Bịch 450g"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Giúp bổ thận, dưỡng huyết, và hỗ trợ làm đen tóc, chống lão hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt macca đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, Macca Lâm Đồng loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt macca đặc sản Langfarm Hũ tobita 200g Bịch mawashi 850g"
      },
      {
        "from": "gpt",
        "value": "373000 đồng, Macca Lâm Đồng loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt macca đặc sản Langfarm Hũ tobita 200g Bịch kraft 250g"
      },
      {
        "from": "gpt",
        "value": "120000 đồng, Macca Lâm Đồng loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt macca đặc sản Langfarm Bịch mawashi 850g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, Macca Lâm Đồng loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt macca đặc sản Langfarm Bịch mawashi 850g"
      },
      {
        "from": "gpt",
        "value": "373000 đồng, Macca Lâm Đồng loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt macca đặc sản Langfarm Bịch mawashi 850g Bịch kraft 250g"
      },
      {
        "from": "gpt",
        "value": "120000 đồng, Macca Lâm Đồng loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt macca đặc sản Langfarm Bịch kraft 250g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, Macca Lâm Đồng loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt macca đặc sản Langfarm Bịch kraft 250g Bịch mawashi 850g"
      },
      {
        "from": "gpt",
        "value": "373000 đồng, Macca Lâm Đồng loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hạt macca đặc sản Langfarm Bịch kraft 250g"
      },
      {
        "from": "gpt",
        "value": "120000 đồng, Macca Lâm Đồng loại 1, thơm béo, dinh dưỡng, healthy"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng chén sấy dẻo đặc sản Langfarm Bịch chân không 300g"
      },
      {
        "from": "gpt",
        "value": "131000 đồng, Hồng Đà Lạt, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng chẻ sấy dẻo đặc sản Langfarm Bịch chân không 300g"
      },
      {
        "from": "gpt",
        "value": "120000 đồng, Hồng Đà Lạt, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Ichidakaki nhập khẩu Nhật Bản Bịch 150g"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Độc quyền tại Langfarm, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp raisu 160g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "209000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp raisu 160g Bịch chân không 250g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "160000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp raisu 160g Hộp vân gỗ 250g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "205000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp raisu 160g Hộp vân gỗ 270g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "241000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Bịch chân không 250g Hộp raisu 160g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "209000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Bịch chân không 250g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "160000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Bịch chân không 250g Hộp vân gỗ 250g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "205000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Bịch chân không 250g Hộp vân gỗ 270g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "241000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp vân gỗ 250g Hộp raisu 160g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "209000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp vân gỗ 250g Bịch chân không 250g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "160000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp vân gỗ 250g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "205000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp vân gỗ 250g Hộp vân gỗ 270g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "241000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp vân gỗ 270g Hộp raisu 160g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "209000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp vân gỗ 270g Bịch chân không 250g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "160000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp vân gỗ 270g Hộp vân gỗ 250g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "205000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng sấy treo Nhật Bản đặc sản Langfarm Hộp vân gỗ 270g Nguyên trái"
      },
      {
        "from": "gpt",
        "value": "241000 đồng, Hồng treo phòng sạch, 14 ngày công phu, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Hồng trà đặc sản Langfarm Lon thiếc 100g"
      },
      {
        "from": "gpt",
        "value": "55000 đồng, Hồng trà Bảo Lộc thượng phẩm, hương vị đậm đà, hương thơm dịu nhẹ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tằm đặc sản Langfarm Hũ tobita 270g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tằm đặc sản Langfarm Hũ tobita 270g Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tằm đặc sản Langfarm Hũ tobita 270g Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "19000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tằm đặc sản Langfarm Bịch kraft 350g Hũ tobita 270g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tằm đặc sản Langfarm Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tằm đặc sản Langfarm Bịch kraft 350g Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "19000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tằm đặc sản Langfarm Túi bạc 180g Hũ tobita 270g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tằm đặc sản Langfarm Túi bạc 180g Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tằm đặc sản Langfarm Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "19000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tây đặc sản Langfarm Hũ tobita 270g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tây đặc sản Langfarm Hũ tobita 270g Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tây đặc sản Langfarm Hũ tobita 270g Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "19000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tây đặc sản Langfarm Bịch kraft 350g Hũ tobita 270g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tây đặc sản Langfarm Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tây đặc sản Langfarm Bịch kraft 350g Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "19000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tây đặc sản Langfarm Túi bạc 180g Hũ tobita 270g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tây đặc sản Langfarm Túi bạc 180g Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo dâu tây đặc sản Langfarm Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "19000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, chua chua ngọt ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo me cay đặc sản Langfarm Hũ tobita 270g"
      },
      {
        "from": "gpt",
        "value": "38000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo me cay đặc sản Langfarm Hũ tobita 270g Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "38000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo me cay đặc sản Langfarm Hũ tobita 270g Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "21000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo me cay đặc sản Langfarm Bịch kraft 350g Hũ tobita 270g"
      },
      {
        "from": "gpt",
        "value": "38000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo me cay đặc sản Langfarm Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "38000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo me cay đặc sản Langfarm Bịch kraft 350g Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "21000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo me cay đặc sản Langfarm Túi bạc 180g Hũ tobita 270g"
      },
      {
        "from": "gpt",
        "value": "38000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo me cay đặc sản Langfarm Túi bạc 180g Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "38000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo me cay đặc sản Langfarm Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "21000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo thập cẩm dẻo đặc sản Langfarm Hũ tobita 300g"
      },
      {
        "from": "gpt",
        "value": "48000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo thập cẩm dẻo đặc sản Langfarm Hũ tobita 300g Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "48000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo thập cẩm dẻo đặc sản Langfarm Hũ tobita 300g Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "26000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo thập cẩm dẻo đặc sản Langfarm Bịch kraft 350g Hũ tobita 300g"
      },
      {
        "from": "gpt",
        "value": "48000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo thập cẩm dẻo đặc sản Langfarm Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "48000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo thập cẩm dẻo đặc sản Langfarm Bịch kraft 350g Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "26000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo thập cẩm dẻo đặc sản Langfarm Túi bạc 180g Hũ tobita 300g"
      },
      {
        "from": "gpt",
        "value": "48000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo thập cẩm dẻo đặc sản Langfarm Túi bạc 180g Bịch kraft 350g"
      },
      {
        "from": "gpt",
        "value": "48000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Kẹo thập cẩm dẻo đặc sản Langfarm Túi bạc 180g"
      },
      {
        "from": "gpt",
        "value": "26000 đồng, Hương Đà Lạt, vị nông sản, an toàn vệ sinh, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang mật sấy nguyên củ đặc sản Langfarm Bịch chân không 320g"
      },
      {
        "from": "gpt",
        "value": "105000 đồng, Khoai lang mật tuyển lựa, ăn vặt nông sản, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang sấy dẻo đặc sản Langfarm Bịch chân không 300g"
      },
      {
        "from": "gpt",
        "value": "94000 đồng, Khoai lang mật tuyển lựa, ăn vặt nông sản, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang tím sấy giòn đặc sản Langfarm Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang tím sấy giòn đặc sản Langfarm Hũ tobita 90g Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang tím sấy giòn đặc sản Langfarm Hũ tobita 90g Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "37000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang tím sấy giòn đặc sản Langfarm Bịch mawashi 350g Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang tím sấy giòn đặc sản Langfarm Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang tím sấy giòn đặc sản Langfarm Bịch mawashi 350g Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "37000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang tím sấy giòn đặc sản Langfarm Bịch kraft 120g Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang tím sấy giòn đặc sản Langfarm Bịch kraft 120g Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang tím sấy giòn đặc sản Langfarm Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "37000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang vàng sấy giòn đặc sản Langfarm Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "32000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang vàng sấy giòn đặc sản Langfarm Hũ tobita 90g Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang vàng sấy giòn đặc sản Langfarm Hũ tobita 90g Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang vàng sấy giòn đặc sản Langfarm Bịch mawashi 350g Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "32000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang vàng sấy giòn đặc sản Langfarm Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang vàng sấy giòn đặc sản Langfarm Bịch mawashi 350g Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang vàng sấy giòn đặc sản Langfarm Bịch kraft 120g Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "32000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang vàng sấy giòn đặc sản Langfarm Bịch kraft 120g Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai lang vàng sấy giòn đặc sản Langfarm Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "36000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai môn sấy giòn đặc sản Langfarm Hũ tobita 80g"
      },
      {
        "from": "gpt",
        "value": "43000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai môn sấy giòn đặc sản Langfarm Hũ tobita 80g Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "129000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai môn sấy giòn đặc sản Langfarm Hũ tobita 80g Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "50000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai môn sấy giòn đặc sản Langfarm Bịch mawashi 350g Hũ tobita 80g"
      },
      {
        "from": "gpt",
        "value": "43000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai môn sấy giòn đặc sản Langfarm Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "129000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai môn sấy giòn đặc sản Langfarm Bịch mawashi 350g Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "50000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai môn sấy giòn đặc sản Langfarm Bịch kraft 120g Hũ tobita 80g"
      },
      {
        "from": "gpt",
        "value": "43000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai môn sấy giòn đặc sản Langfarm Bịch kraft 120g Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "129000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khoai môn sấy giòn đặc sản Langfarm Bịch kraft 120g"
      },
      {
        "from": "gpt",
        "value": "50000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khô gà cay lá chanh đặc sản Langfarm Hũ tobita 100g"
      },
      {
        "from": "gpt",
        "value": "54000 đồng, Thịt ức gà tươi, đậm đà thơm ngon, giòn rụm, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khô gà cay lá chanh đặc sản Langfarm Hũ tobita 100g Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "142000 đồng, Thịt ức gà tươi, đậm đà thơm ngon, giòn rụm, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khô gà cay lá chanh đặc sản Langfarm Hũ tobita 100g Bịch kraft 125g"
      },
      {
        "from": "gpt",
        "value": "57000 đồng, Thịt ức gà tươi, đậm đà thơm ngon, giòn rụm, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khô gà cay lá chanh đặc sản Langfarm Bịch mawashi 350g Hũ tobita 100g"
      },
      {
        "from": "gpt",
        "value": "54000 đồng, Thịt ức gà tươi, đậm đà thơm ngon, giòn rụm, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khô gà cay lá chanh đặc sản Langfarm Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "142000 đồng, Thịt ức gà tươi, đậm đà thơm ngon, giòn rụm, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khô gà cay lá chanh đặc sản Langfarm Bịch mawashi 350g Bịch kraft 125g"
      },
      {
        "from": "gpt",
        "value": "57000 đồng, Thịt ức gà tươi, đậm đà thơm ngon, giòn rụm, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khô gà cay lá chanh đặc sản Langfarm Bịch kraft 125g Hũ tobita 100g"
      },
      {
        "from": "gpt",
        "value": "54000 đồng, Thịt ức gà tươi, đậm đà thơm ngon, giòn rụm, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khô gà cay lá chanh đặc sản Langfarm Bịch kraft 125g Bịch mawashi 350g"
      },
      {
        "from": "gpt",
        "value": "142000 đồng, Thịt ức gà tươi, đậm đà thơm ngon, giòn rụm, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khô gà cay lá chanh đặc sản Langfarm Bịch kraft 125g"
      },
      {
        "from": "gpt",
        "value": "57000 đồng, Thịt ức gà tươi, đậm đà thơm ngon, giòn rụm, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khổ qua sấy khô đặc sản Langfarm Hũ yama 130g"
      },
      {
        "from": "gpt",
        "value": "68000 đồng, Giúp hỗ trợ giảm đường huyết, thanh nhiệt và tăng cường sức khỏe tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khổ qua sấy khô đặc sản Langfarm Hũ yama 130g Bịch 225g"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Giúp hỗ trợ giảm đường huyết, thanh nhiệt và tăng cường sức khỏe tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khổ qua sấy khô đặc sản Langfarm Bịch 225g Hũ yama 130g"
      },
      {
        "from": "gpt",
        "value": "68000 đồng, Giúp hỗ trợ giảm đường huyết, thanh nhiệt và tăng cường sức khỏe tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khổ qua sấy khô đặc sản Langfarm Bịch 225g"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Giúp hỗ trợ giảm đường huyết, thanh nhiệt và tăng cường sức khỏe tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khuếch tán tinh dầu núi đá trắng, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "139000 đồng, Thiết kế đẹp, chất liệu tốt, hiệu quả cao, an toàn cho người dùng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khuếch tán tinh dầu núi đá đen, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "139000 đồng, Thiết kế đẹp, chất liệu tốt, hiệu quả cao, an toàn cho người dùng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khuếch tán tinh dầu gỗ dẻ gai, cái, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "19000 đồng, Thiết kế đẹp, chất liệu tốt, hiệu quả cao, an toàn cho người dùng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khuếch tán tinh dầu núi đá trắng, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "139000 đồng, Thiết kế đẹp, chất liệu tốt, hiệu quả cao, an toàn cho người dùng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khuếch tán tinh dầu núi đá đen, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "139000 đồng, Thiết kế đẹp, chất liệu tốt, hiệu quả cao, an toàn cho người dùng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khuếch tán tinh dầu gỗ dẻ gai, cái, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "19000 đồng, Thiết kế đẹp, chất liệu tốt, hiệu quả cao, an toàn cho người dùng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khuếch tán tinh dầu núi đá trắng, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "139000 đồng, Thiết kế đẹp, chất liệu tốt, hiệu quả cao, an toàn cho người dùng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khuếch tán tinh dầu núi đá đen, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "139000 đồng, Thiết kế đẹp, chất liệu tốt, hiệu quả cao, an toàn cho người dùng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Khuếch tán tinh dầu gỗ dẻ gai, cái, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "19000 đồng, Thiết kế đẹp, chất liệu tốt, hiệu quả cao, an toàn cho người dùng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Long nhãn sấy dẻo đặc sản Langfarm Hũ tobita 150g"
      },
      {
        "from": "gpt",
        "value": "81000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Long nhãn sấy dẻo đặc sản Langfarm Hũ tobita 150g Bịch kraft 180g"
      },
      {
        "from": "gpt",
        "value": "95000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Long nhãn sấy dẻo đặc sản Langfarm Bịch kraft 180g Hũ tobita 150g"
      },
      {
        "from": "gpt",
        "value": "81000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Long nhãn sấy dẻo đặc sản Langfarm Bịch kraft 180g"
      },
      {
        "from": "gpt",
        "value": "95000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mận sấy dẻo đặc sản Langfarm Hũ tobita 250g"
      },
      {
        "from": "gpt",
        "value": "79000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mận sấy dẻo đặc sản Langfarm Hũ tobita 250g Bịch kraft 300g"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mận sấy dẻo đặc sản Langfarm Bịch kraft 300g Hũ tobita 250g"
      },
      {
        "from": "gpt",
        "value": "79000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mận sấy dẻo đặc sản Langfarm Bịch kraft 300g"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Máy xay cà phê cầm tay, cái, mẫu retro rouet, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "549000 đồng, Máy xay đẹp, vật liệu tốt, dễ sử dụng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mít sấy giòn đặc sản Langfarm Bịch mawashi 225g"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mít sấy giòn đặc sản Langfarm Bịch mawashi 225g Hũ tobita 60g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mít sấy giòn đặc sản Langfarm Bịch mawashi 225g Bịch kraft 80g"
      },
      {
        "from": "gpt",
        "value": "35000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mít sấy giòn đặc sản Langfarm Hũ tobita 60g Bịch mawashi 225g"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mít sấy giòn đặc sản Langfarm Hũ tobita 60g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mít sấy giòn đặc sản Langfarm Hũ tobita 60g Bịch kraft 80g"
      },
      {
        "from": "gpt",
        "value": "35000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mít sấy giòn đặc sản Langfarm Bịch kraft 80g Bịch mawashi 225g"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mít sấy giòn đặc sản Langfarm Bịch kraft 80g Hũ tobita 60g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mít sấy giòn đặc sản Langfarm Bịch kraft 80g"
      },
      {
        "from": "gpt",
        "value": "35000 đồng, Ăn vặt nông sản, giòn tan trong miệng, hàng loại A"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực bơ tỏi ăn liền đặc sản Yumsea x Langfarm - -"
      },
      {
        "from": "gpt",
        "value": "267000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực hấp xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch kraft 90g"
      },
      {
        "from": "gpt",
        "value": "78000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực hấp xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch kraft 90g Bịch mawashi 260g"
      },
      {
        "from": "gpt",
        "value": "212000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực hấp xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch kraft 90g Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực hấp xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch mawashi 260g Bịch kraft 90g"
      },
      {
        "from": "gpt",
        "value": "78000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực hấp xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch mawashi 260g"
      },
      {
        "from": "gpt",
        "value": "212000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực hấp xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch mawashi 260g Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực hấp xé sợi ăn liền đặc sản Yumsea x Langfarm Hũ tobita 90g Bịch kraft 90g"
      },
      {
        "from": "gpt",
        "value": "78000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực hấp xé sợi ăn liền đặc sản Yumsea x Langfarm Hũ tobita 90g Bịch mawashi 260g"
      },
      {
        "from": "gpt",
        "value": "212000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực hấp xé sợi ăn liền đặc sản Yumsea x Langfarm Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực khô xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch mawashi 260g"
      },
      {
        "from": "gpt",
        "value": "215000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực khô xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch mawashi 260g Hũ tobita 65g"
      },
      {
        "from": "gpt",
        "value": "71000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực khô xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch mawashi 260g Bịch kraft 70g"
      },
      {
        "from": "gpt",
        "value": "61000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực khô xé sợi ăn liền đặc sản Yumsea x Langfarm Hũ tobita 65g Bịch mawashi 260g"
      },
      {
        "from": "gpt",
        "value": "215000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực khô xé sợi ăn liền đặc sản Yumsea x Langfarm Hũ tobita 65g"
      },
      {
        "from": "gpt",
        "value": "71000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực khô xé sợi ăn liền đặc sản Yumsea x Langfarm Hũ tobita 65g Bịch kraft 70g"
      },
      {
        "from": "gpt",
        "value": "61000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực khô xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch kraft 70g Bịch mawashi 260g"
      },
      {
        "from": "gpt",
        "value": "215000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực khô xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch kraft 70g Hũ tobita 65g"
      },
      {
        "from": "gpt",
        "value": "71000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực khô xé sợi ăn liền đặc sản Yumsea x Langfarm Bịch kraft 70g"
      },
      {
        "from": "gpt",
        "value": "61000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực sốt phô mai ăn liền đặc sản Yumsea x Langfarm Hũ tobita 130g"
      },
      {
        "from": "gpt",
        "value": "164000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực sốt phô mai ăn liền đặc sản Yumsea x Langfarm Hũ tobita 130g Bịch kraft 140g"
      },
      {
        "from": "gpt",
        "value": "162000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực sốt phô mai ăn liền đặc sản Yumsea x Langfarm Bịch kraft 140g Hũ tobita 130g"
      },
      {
        "from": "gpt",
        "value": "164000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mực sốt phô mai ăn liền đặc sản Yumsea x Langfarm Bịch kraft 140g"
      },
      {
        "from": "gpt",
        "value": "162000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mứt nho đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mứt nho đặc sản Langfarm Hũ tobita 200g Bịch kraft 250g"
      },
      {
        "from": "gpt",
        "value": "92000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mứt nho đặc sản Langfarm Bịch kraft 250g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Mứt nho đặc sản Langfarm Bịch kraft 250g"
      },
      {
        "from": "gpt",
        "value": "92000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nấm linh chi đỏ Đà Lạt đặc sản Langfarm Bịch 225g"
      },
      {
        "from": "gpt",
        "value": "329000 đồng, Giúp tăng cường hệ miễn dịch, cải thiện sức khỏe tổng thể và hỗ trợ chống lão hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Hộp thiếc 120g Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Panther collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Elegant collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Leaf story collection Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Winter moss"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Vanilla"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Hộp thiếc 120g Orange flower"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Strawberry and mint"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Nectarine and bamboo"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Passion fruit"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Panther collection Hamimelon and pear"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Saffon and sandal wood"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Seasalt breeze"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection Blanc velvet"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Elegant collection City of light"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Apple cider"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection White tea"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Peach"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Leaf story collection Ocean"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Verbena"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Lavender"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Lemon & sage"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nến thơm hộp thiếc hoa văn Dalat Retro x Langfarm Four seasons collection Rose"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Ngũ hạt thập cẩm đặc sản Langfarm Hũ tobita 135g"
      },
      {
        "from": "gpt",
        "value": "42000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Ngũ hạt thập cẩm đặc sản Langfarm Hũ tobita 135g Bịch kraft 185g"
      },
      {
        "from": "gpt",
        "value": "48000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Ngũ hạt thập cẩm đặc sản Langfarm Bịch kraft 185g Hũ tobita 135g"
      },
      {
        "from": "gpt",
        "value": "42000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Ngũ hạt thập cẩm đặc sản Langfarm Bịch kraft 185g"
      },
      {
        "from": "gpt",
        "value": "48000 đồng, Ăn văt dinh dưỡng, giòn ngon, bắt miệng, an toàn vệ sinh"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Nước cốt chanh dây đặc sản Dâu Farm x Langfarm Chai antique 180ml"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, 100% chanh dây tươi Đà Lạt, công nghệ chân không, cô đặc gấp 3, hương vị tươi ngon"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Ổi dẻo đặc sản Langfarm Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Ổi dẻo đặc sản Langfarm Bịch kraft 225g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "68000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Ổi dẻo đặc sản Langfarm Hũ tobita 200g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Ổi dẻo đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "68000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic silver, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro red, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro blue, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro green, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro orange, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic black, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu rose gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu bronze gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu amber gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic silver, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro red, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro blue, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro green, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro orange, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic black, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu rose gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu bronze gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu amber gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic silver, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro red, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro blue, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro green, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro orange, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic black, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu rose gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu bronze gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu amber gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic silver, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro red, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro blue, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro green, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro orange, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic black, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu rose gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu bronze gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu amber gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic silver, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro red, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro blue, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro green, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro orange, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic black, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu rose gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu bronze gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu amber gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic silver, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro red, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro blue, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro green, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro orange, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic black, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu rose gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu bronze gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu amber gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic silver, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro red, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro blue, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro green, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro orange, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic black, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu rose gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu bronze gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu amber gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic silver, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro red, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro blue, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro green, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro orange, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic black, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu rose gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu bronze gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu amber gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic silver, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "99000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro red, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro blue, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro green, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu bắn cát màu retro orange, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu metallic black, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu rose gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu bronze gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Phin cà phê nhôm anode, mẫu cào xước màu amber gold, hộp, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "149000 đồng, Loại phin cà phê nhôm cao cấp nhất trên thị trường"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Rong biển kẹp ngũ cốc đặc sản Yumsea x Langfarm Hũ tobita 110g"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Bữa ăn tiện lợi, đầy đủ dinh dưỡng, giòn ngon khó cưỡng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương champagne fruit wine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương coco, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương neroli bergamot, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương sandalwood sweet orange, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương green tea, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương english pear freesia, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương golden temptation, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương midnight lancome, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương french vanilla, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương rose, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sáp thơm hoa khô handmade, hương jasmine, 65g, hộp, mẫu kawamura, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "87000 đồng, Sáp ong & đậu nành tự nhiên, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà bản giao hưởng cà phê, 4 món, hộp, mẫu quai da, Langfarm x Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "659000 đồng, Phin, ly, máy xay và cà phê hạt đặc sản Lâm Đồng chất lượng cao"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà Langfarm De Amour, 13 món, hộp, Langfarm"
      },
      {
        "from": "gpt",
        "value": "285000 đồng, Hương nông sản, vị núi rừng, gồm 13 món (5 màu ngẫu nhiên)"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, leaf story, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, four season, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, elegant, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, black panther, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, leaf story, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, four season, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, elegant, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, black panther, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, leaf story, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, four season, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, elegant, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, black panther, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, leaf story, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, four season, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, elegant, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà nến thơm, black panther, hộp, mẫu Ana, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "596000 đồng, Sáp đậu nành tự nhiên, tinh dầu xịn, thơm nhẹ nhàng, hoa khô handmade"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Retro"
      },
      {
        "from": "gpt",
        "value": "1990000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Gourmet"
      },
      {
        "from": "gpt",
        "value": "1799000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Luna"
      },
      {
        "from": "gpt",
        "value": "1099000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Etsy"
      },
      {
        "from": "gpt",
        "value": "659000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Ginko"
      },
      {
        "from": "gpt",
        "value": "499000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Retro"
      },
      {
        "from": "gpt",
        "value": "1990000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Gourmet"
      },
      {
        "from": "gpt",
        "value": "1799000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Luna"
      },
      {
        "from": "gpt",
        "value": "1099000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Etsy"
      },
      {
        "from": "gpt",
        "value": "659000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Ginko"
      },
      {
        "from": "gpt",
        "value": "499000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Retro"
      },
      {
        "from": "gpt",
        "value": "1990000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Gourmet"
      },
      {
        "from": "gpt",
        "value": "1799000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Luna"
      },
      {
        "from": "gpt",
        "value": "1099000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Etsy"
      },
      {
        "from": "gpt",
        "value": "659000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Ginko"
      },
      {
        "from": "gpt",
        "value": "499000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Retro"
      },
      {
        "from": "gpt",
        "value": "1990000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Gourmet"
      },
      {
        "from": "gpt",
        "value": "1799000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Luna"
      },
      {
        "from": "gpt",
        "value": "1099000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Etsy"
      },
      {
        "from": "gpt",
        "value": "659000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Ginko"
      },
      {
        "from": "gpt",
        "value": "499000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Retro"
      },
      {
        "from": "gpt",
        "value": "1990000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Gourmet"
      },
      {
        "from": "gpt",
        "value": "1799000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Luna"
      },
      {
        "from": "gpt",
        "value": "1099000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Etsy"
      },
      {
        "from": "gpt",
        "value": "659000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set quà tết đặc sản Langfarm 2025 collection Set Langfarm Ginko"
      },
      {
        "from": "gpt",
        "value": "499000 đồng, Hương Đà Lạt, vị nông sản, thiết kế tinh tế, hoàn thiện tinh xảo"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set tổ yến thiên nhiên thượng hạng Yumsea x Langfarm Showa 50g"
      },
      {
        "from": "gpt",
        "value": "2510000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set tổ yến thiên nhiên thượng hạng Yumsea x Langfarm Showa 50g Origami 20g"
      },
      {
        "from": "gpt",
        "value": "1032000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set tổ yến thiên nhiên thượng hạng Yumsea x Langfarm Origami 20g Showa 50g"
      },
      {
        "from": "gpt",
        "value": "2510000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Set tổ yến thiên nhiên thượng hạng Yumsea x Langfarm Origami 20g"
      },
      {
        "from": "gpt",
        "value": "1032000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Sữa chua yến sấy thăng hoa đặc sản Yumsea x Langfarm Hộp pillow 2 bánh"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Tổ yến nguyên chất, bánh ngon, bổ dưỡng, tan trong miệng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Táo sấy khô đặc sản Langfarm Hũ tobita 250g"
      },
      {
        "from": "gpt",
        "value": "81000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Táo sấy khô đặc sản Langfarm Hũ tobita 250g Bịch kraft 300g"
      },
      {
        "from": "gpt",
        "value": "90000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Táo sấy khô đặc sản Langfarm Bịch kraft 300g Hũ tobita 250g"
      },
      {
        "from": "gpt",
        "value": "81000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Táo sấy khô đặc sản Langfarm Bịch kraft 300g"
      },
      {
        "from": "gpt",
        "value": "90000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thanh long sấy dẻo đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thanh long sấy dẻo đặc sản Langfarm Hũ tobita 200g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thanh long sấy dẻo đặc sản Langfarm Bịch kraft 225g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thanh long sấy dẻo đặc sản Langfarm Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Hũ tobita 90g Bịch mawashi 400g"
      },
      {
        "from": "gpt",
        "value": "95000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Hũ tobita 90g Bịch kraft 100g"
      },
      {
        "from": "gpt",
        "value": "32000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Hũ tobita 90g Bịch nhãn kẹp 450g"
      },
      {
        "from": "gpt",
        "value": "100000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch mawashi 400g Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch mawashi 400g"
      },
      {
        "from": "gpt",
        "value": "95000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch mawashi 400g Bịch kraft 100g"
      },
      {
        "from": "gpt",
        "value": "32000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch mawashi 400g Bịch nhãn kẹp 450g"
      },
      {
        "from": "gpt",
        "value": "100000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch kraft 100g Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch kraft 100g Bịch mawashi 400g"
      },
      {
        "from": "gpt",
        "value": "95000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch kraft 100g"
      },
      {
        "from": "gpt",
        "value": "32000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch kraft 100g Bịch nhãn kẹp 450g"
      },
      {
        "from": "gpt",
        "value": "100000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch nhãn kẹp 450g Hũ tobita 90g"
      },
      {
        "from": "gpt",
        "value": "33000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch nhãn kẹp 450g Bịch mawashi 400g"
      },
      {
        "from": "gpt",
        "value": "95000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch nhãn kẹp 450g Bịch kraft 100g"
      },
      {
        "from": "gpt",
        "value": "32000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thập cẩm sấy giòn đặc sản Langfarm Bịch nhãn kẹp 450g"
      },
      {
        "from": "gpt",
        "value": "100000 đồng, Ăn vạt nông sản, giòn tan từng miếng, trọn vị thiên nhiên"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thơm sấy dẻo đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thơm sấy dẻo đặc sản Langfarm Hũ tobita 200g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "101000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thơm sấy dẻo đặc sản Langfarm Bịch kraft 225g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "91000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Thơm sấy dẻo đặc sản Langfarm Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "101000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hương thảo rosemary, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bạc hà peppermint, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, sả chanh lemongrass, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "88000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, chanh lemon, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, bưởi hồng grapefruit, 10ml, hộp, mẫu minimal Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "96000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, cam orange, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "84000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, tràm gió cajeput, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "80000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, gỗ hồng rose wood, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "97000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, thông đỏ red pine, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "89000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, oải hương pháp lavender, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "110000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, ngọc lan tây ylang ylang, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tinh dầu nguyên chất, hoa hồng rose, 10ml, hộp, mẫu minimal, Dalat Retro"
      },
      {
        "from": "gpt",
        "value": "189000 đồng, Thiên nhiên nguyên chất, hương thơm tự nhiên, nhẹ nhàng, cảm xúc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Tôm rim rau củ ăn liền đặc sản Yumsea x Langfarm Hũ tobita 130g"
      },
      {
        "from": "gpt",
        "value": "113000 đồng, Ăn vặt đất biển, đậm đà thơm ngon, an toàn vệ sinh, ăn là ghiền"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp truyền thống 60 tép"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Bịch truyền thống 100 tép"
      },
      {
        "from": "gpt",
        "value": "119000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp truyền thống 60 tép"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Bịch truyền thống 100 tép"
      },
      {
        "from": "gpt",
        "value": "119000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp truyền thống 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp truyền thống 60 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp truyền thống 60 tép"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Hộp truyền thống 60 tép Bịch truyền thống 100 tép"
      },
      {
        "from": "gpt",
        "value": "119000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Bịch truyền thống 100 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Bịch truyền thống 100 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Bịch truyền thống 100 tép Hộp truyền thống 60 tép"
      },
      {
        "from": "gpt",
        "value": "98000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà atisô túi lọc thảo mộc Thái Bảo x Langfarm Bịch truyền thống 100 tép"
      },
      {
        "from": "gpt",
        "value": "119000 đồng, Giúp mát gan, thanh lọc cơ thể, đẹp da"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà bồ công anh đặc sản Langfarm Hũ yama 70g"
      },
      {
        "from": "gpt",
        "value": "72000 đồng, Giúp thanh nhiệt, giải độc gan và hỗ trợ điều trị viêm loét dạ dày"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà cỏ ngọt túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "60000 đồng, Giúp tiêu khát, lợi tiểu, hạ huyết áp"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà cỏ ngọt túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Giúp tiêu khát, lợi tiểu, hạ huyết áp"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà cỏ ngọt túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "60000 đồng, Giúp tiêu khát, lợi tiểu, hạ huyết áp"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà cỏ ngọt túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Giúp tiêu khát, lợi tiểu, hạ huyết áp"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà đắng đặc sản Langfarm Hũ yama 130g"
      },
      {
        "from": "gpt",
        "value": "127000 đồng, Giúp thanh nhiệt, giảm mỡ máu và hỗ trợ kiểm soát đường huyết"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà đắng túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp thanh lọc cơ thể, hỗ trợ tiêu hóa và giảm cholesterol"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà đắng túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "51000 đồng, Giúp thanh lọc cơ thể, hỗ trợ tiêu hóa và giảm cholesterol"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà đắng túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp thanh lọc cơ thể, hỗ trợ tiêu hóa và giảm cholesterol"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà đắng túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "51000 đồng, Giúp thanh lọc cơ thể, hỗ trợ tiêu hóa và giảm cholesterol"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà diệp hạ châu túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp mát gan, hỗ trợ giải độc gan và bảo vệ chức năng gan"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà diệp hạ châu túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "58000 đồng, Giúp mát gan, hỗ trợ giải độc gan và bảo vệ chức năng gan"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà diệp hạ châu túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp mát gan, hỗ trợ giải độc gan và bảo vệ chức năng gan"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà diệp hạ châu túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "58000 đồng, Giúp mát gan, hỗ trợ giải độc gan và bảo vệ chức năng gan"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà đông trùng hạ thảo túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "362000 đồng, Giúp tăng cường sức đề kháng, bồi bổ sức khỏe và hỗ trợ cải thiện chức năng hô hấp"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà giảo cổ lam đặc sản Langfarm Hũ yama 90g"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, Giúp hỗ trợ giảm cholesterol, kiểm soát đường huyết và tăng cường sức khỏe tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà gừng hoà tan đặc sản Thái Bảo x Langfarm Hộp 5 túi"
      },
      {
        "from": "gpt",
        "value": "24000 đồng, Tốt cho người đau bụng lạnh, khó tiêu, ho suyễn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà gừng hoà tan đặc sản Thái Bảo x Langfarm Hộp 5 túi Bịch 22 túi"
      },
      {
        "from": "gpt",
        "value": "86000 đồng, Tốt cho người đau bụng lạnh, khó tiêu, ho suyễn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà gừng hoà tan đặc sản Thái Bảo x Langfarm Bịch 22 túi Hộp 5 túi"
      },
      {
        "from": "gpt",
        "value": "24000 đồng, Tốt cho người đau bụng lạnh, khó tiêu, ho suyễn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà gừng hoà tan đặc sản Thái Bảo x Langfarm Bịch 22 túi"
      },
      {
        "from": "gpt",
        "value": "86000 đồng, Tốt cho người đau bụng lạnh, khó tiêu, ho suyễn"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà gừng túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp giảm cảm lạnh, tăng cường hệ miễn dịch và hỗ trợ tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà gừng túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "51000 đồng, Giúp giảm cảm lạnh, tăng cường hệ miễn dịch và hỗ trợ tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà gừng túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp giảm cảm lạnh, tăng cường hệ miễn dịch và hỗ trợ tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà gừng túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "51000 đồng, Giúp giảm cảm lạnh, tăng cường hệ miễn dịch và hỗ trợ tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà hà thủ ô túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Giúp bổ thận, đen tóc và tăng cường sức khoẻ tóc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà hà thủ ô túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "68000 đồng, Giúp bổ thận, đen tóc và tăng cường sức khoẻ tóc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà hà thủ ô túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Giúp bổ thận, đen tóc và tăng cường sức khoẻ tóc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà hà thủ ô túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "68000 đồng, Giúp bổ thận, đen tóc và tăng cường sức khoẻ tóc"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà hoa cúc túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "104000 đồng, Giúp thư giãn, giảm căng thẳng và hỗ trợ giấc ngủ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà hoa cúc túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "75000 đồng, Giúp thư giãn, giảm căng thẳng và hỗ trợ giấc ngủ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà hoa cúc túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "104000 đồng, Giúp thư giãn, giảm căng thẳng và hỗ trợ giấc ngủ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà hoa cúc túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "75000 đồng, Giúp thư giãn, giảm căng thẳng và hỗ trợ giấc ngủ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà khổ qua túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp hạ đường huyết, hỗ trợ kiểm soát tiểu đường và thanh nhiệt cơ thể"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà khổ qua túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Giúp hạ đường huyết, hỗ trợ kiểm soát tiểu đường và thanh nhiệt cơ thể"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà khổ qua túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp hạ đường huyết, hỗ trợ kiểm soát tiểu đường và thanh nhiệt cơ thể"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà khổ qua túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "63000 đồng, Giúp hạ đường huyết, hỗ trợ kiểm soát tiểu đường và thanh nhiệt cơ thể"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà lá dâu đặc sản Langfarm Hũ yama 150g"
      },
      {
        "from": "gpt",
        "value": "92000 đồng, Giúp thanh nhiệt, hỗ trợ giảm ho, và tăng cường sức khỏe đường hô hấp"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà lài đặc sản Langfarm Hộp cửa sổ đỏ 100g"
      },
      {
        "from": "gpt",
        "value": "59000 đồng, Bản giao hưởng của lá trà hảo hạng vào hương hoa lài thơm ngát, hậu ngọt thanh, hương dịu nhẹ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà lá sen đặc sản Langfarm Hũ yama 150g"
      },
      {
        "from": "gpt",
        "value": "83000 đồng, Hỗ trợ giảm cân, thanh lọc cơ thể, và cải thiện giấc ngủ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà linh chi túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "208000 đồng, Giúp tăng cường hệ miễn dịch, giảm căng thẳng và hỗ trợ sức khỏe tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà linh chi túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "126000 đồng, Giúp tăng cường hệ miễn dịch, giảm căng thẳng và hỗ trợ sức khỏe tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà linh chi túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "208000 đồng, Giúp tăng cường hệ miễn dịch, giảm căng thẳng và hỗ trợ sức khỏe tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà linh chi túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "126000 đồng, Giúp tăng cường hệ miễn dịch, giảm căng thẳng và hỗ trợ sức khỏe tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà nhân trần túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Giúp mát gan, giải độc và hỗ trợ tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà nhân trần túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "55000 đồng, Giúp mát gan, giải độc và hỗ trợ tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà nhân trần túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Giúp mát gan, giải độc và hỗ trợ tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà nhân trần túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "55000 đồng, Giúp mát gan, giải độc và hỗ trợ tiêu hóa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà nụ vối túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp hỗ trợ tiêu hóa, giảm mỡ máu và kiểm soát đường huyết"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà nụ vối túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "58000 đồng, Giúp hỗ trợ tiêu hóa, giảm mỡ máu và kiểm soát đường huyết"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà nụ vối túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp hỗ trợ tiêu hóa, giảm mỡ máu và kiểm soát đường huyết"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà nụ vối túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "58000 đồng, Giúp hỗ trợ tiêu hóa, giảm mỡ máu và kiểm soát đường huyết"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà oolong đặc sản Langfarm Lon thiếc 150g"
      },
      {
        "from": "gpt",
        "value": "204000 đồng, Trà oolong Cầu Đất thượng phẩm, hương thơm thanh mát, vị ngọt hậu nhẹ nhang"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà oolong đặc sản Langfarm Lon thiếc 150g Hộp cửa sổ đỏ 150g"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Trà oolong Cầu Đất thượng phẩm, hương thơm thanh mát, vị ngọt hậu nhẹ nhang"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà oolong đặc sản Langfarm Hộp cửa sổ đỏ 150g Lon thiếc 150g"
      },
      {
        "from": "gpt",
        "value": "204000 đồng, Trà oolong Cầu Đất thượng phẩm, hương thơm thanh mát, vị ngọt hậu nhẹ nhang"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà oolong đặc sản Langfarm Hộp cửa sổ đỏ 150g"
      },
      {
        "from": "gpt",
        "value": "186000 đồng, Trà oolong Cầu Đất thượng phẩm, hương thơm thanh mát, vị ngọt hậu nhẹ nhang"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà oolong đỏ đặc sản Langfarm Lon thiếc 150g"
      },
      {
        "from": "gpt",
        "value": "120000 đồng, Trà oolong đỏ Lâm Đồng thượng phẩm, hương thơm, hậu ngọt, nước đỏ hổ phách"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà oolong túi lọc đặc sản Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "119000 đồng, Giúp tăng cường trao đổi chất, hỗ trợ giảm cân và tốt cho tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà oolong túi lọc đặc sản Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "68000 đồng, Giúp tăng cường trao đổi chất, hỗ trợ giảm cân và tốt cho tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà oolong túi lọc đặc sản Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "119000 đồng, Giúp tăng cường trao đổi chất, hỗ trợ giảm cân và tốt cho tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà oolong túi lọc đặc sản Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "68000 đồng, Giúp tăng cường trao đổi chất, hỗ trợ giảm cân và tốt cho tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà râu bắp túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp lợi tiểu, hỗ trợ thải độc và giảm viêm đường tiết niệu"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà râu bắp túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "55000 đồng, Giúp lợi tiểu, hỗ trợ thải độc và giảm viêm đường tiết niệu"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà râu bắp túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp lợi tiểu, hỗ trợ thải độc và giảm viêm đường tiết niệu"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà râu bắp túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "55000 đồng, Giúp lợi tiểu, hỗ trợ thải độc và giảm viêm đường tiết niệu"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà sâm túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "104000 đồng, Giúp bồi bổ khí huyết, tăng cường sức đề kháng và cải thiện sức khỏe tổng thể"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà sâm túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Giúp bồi bổ khí huyết, tăng cường sức đề kháng và cải thiện sức khỏe tổng thể"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà sâm túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "104000 đồng, Giúp bồi bổ khí huyết, tăng cường sức đề kháng và cải thiện sức khỏe tổng thể"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà sâm túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "82000 đồng, Giúp bồi bổ khí huyết, tăng cường sức đề kháng và cải thiện sức khỏe tổng thể"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà sencha hoa cúc Matchi Matcha x Langfarm Hộp stamp 82g"
      },
      {
        "from": "gpt",
        "value": "107000 đồng, Lá trà sencha nhập khẩu từ Kagoshima, Nhật Bản, bản giao hưởng của trà và hoa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà sencha hoa hồng Matchi Matcha x Langfarm Hộp stamp 86g"
      },
      {
        "from": "gpt",
        "value": "109000 đồng, Lá trà sencha nhập khẩu từ Kagoshima, Nhật Bản, bản giao hưởng của trà và hoa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà sencha hoa lài Matchi Matcha x Langfarm Hộp stamp 80g"
      },
      {
        "from": "gpt",
        "value": "105000 đồng, Lá trà sencha nhập khẩu từ Kagoshima, Nhật Bản, bản giao hưởng của trà và hoa"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà shan tuyết đặc sản Langfarm Lon thiếc 80g"
      },
      {
        "from": "gpt",
        "value": "219000 đồng, Trà shan tuyết cổ thụ Suối Giàng thượng phẩm, tinh hoa cây trà cổ thụ hàng trăm năm tuổi"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà thanh nhiệt túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Giúp thanh nhiệt, sáng mắt, nhuận tràng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà thanh nhiệt túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "54000 đồng, Giúp thanh nhiệt, sáng mắt, nhuận tràng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà thanh nhiệt túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Giúp thanh nhiệt, sáng mắt, nhuận tràng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà thanh nhiệt túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "54000 đồng, Giúp thanh nhiệt, sáng mắt, nhuận tràng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà tim sen túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Giúp an thần, cải thiện giấc ngủ và giảm căng thẳng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà tim sen túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "60000 đồng, Giúp an thần, cải thiện giấc ngủ và giảm căng thẳng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà tim sen túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "85000 đồng, Giúp an thần, cải thiện giấc ngủ và giảm căng thẳng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà tim sen túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "60000 đồng, Giúp an thần, cải thiện giấc ngủ và giảm căng thẳng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà trái nhàu túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp nhuận tràng, lợi tiểu, điều kinh, hoạt huyết"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà trái nhàu túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "50000 đồng, Giúp nhuận tràng, lợi tiểu, điều kinh, hoạt huyết"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà trái nhàu túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "77000 đồng, Giúp nhuận tràng, lợi tiểu, điều kinh, hoạt huyết"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà trái nhàu túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "50000 đồng, Giúp nhuận tràng, lợi tiểu, điều kinh, hoạt huyết"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà trinh nữ hoàng cung túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "117000 đồng, Hỗ trợ phòng ngừa u xơ, tăng cường sức khỏe sinh sản và giảm viêm nhiễm"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà trinh nữ hoàng cung túi lọc thảo mộc Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "60000 đồng, Hỗ trợ phòng ngừa u xơ, tăng cường sức khỏe sinh sản và giảm viêm nhiễm"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà trinh nữ hoàng cung túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "117000 đồng, Hỗ trợ phòng ngừa u xơ, tăng cường sức khỏe sinh sản và giảm viêm nhiễm"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà trinh nữ hoàng cung túi lọc thảo mộc Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "60000 đồng, Hỗ trợ phòng ngừa u xơ, tăng cường sức khỏe sinh sản và giảm viêm nhiễm"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà xanh đặc sản Langfarm Hộp cửa sổ đỏ 100g"
      },
      {
        "from": "gpt",
        "value": "72000 đồng, Trà xanh Thái Nguyên thượng phẩm, lá trà tươi non, chọn lọc kỹ càng, hậu ngọt quyến rũ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà xanh đặc sản Langfarm Hộp cửa sổ đỏ 100g Lon thiếc 90g"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, Trà xanh Thái Nguyên thượng phẩm, lá trà tươi non, chọn lọc kỹ càng, hậu ngọt quyến rũ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà xanh đặc sản Langfarm Lon thiếc 90g Hộp cửa sổ đỏ 100g"
      },
      {
        "from": "gpt",
        "value": "72000 đồng, Trà xanh Thái Nguyên thượng phẩm, lá trà tươi non, chọn lọc kỹ càng, hậu ngọt quyến rũ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà xanh đặc sản Langfarm Lon thiếc 90g"
      },
      {
        "from": "gpt",
        "value": "103000 đồng, Trà xanh Thái Nguyên thượng phẩm, lá trà tươi non, chọn lọc kỹ càng, hậu ngọt quyến rũ"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà xanh Nhật Bản Langfarm Lon thiếc 100g"
      },
      {
        "from": "gpt",
        "value": "105000 đồng, Trà sencha nhập khẩu từ Kagoshima, Nhật Bản, hương vị trà sencha thơm ngon đặc trưng"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà xanh túi lọc đặc sản Thái Bảo x Langfarm Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Giúp tăng cường trao đổi chất, hỗ trợ giảm cân và tốt cho tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà xanh túi lọc đặc sản Thái Bảo x Langfarm Hộp pastel 60 tép Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "55000 đồng, Giúp tăng cường trao đổi chất, hỗ trợ giảm cân và tốt cho tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà xanh túi lọc đặc sản Thái Bảo x Langfarm Hộp kraft 25 tép Hộp pastel 60 tép"
      },
      {
        "from": "gpt",
        "value": "69000 đồng, Giúp tăng cường trao đổi chất, hỗ trợ giảm cân và tốt cho tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Trà xanh túi lọc đặc sản Thái Bảo x Langfarm Hộp kraft 25 tép"
      },
      {
        "from": "gpt",
        "value": "55000 đồng, Giúp tăng cường trao đổi chất, hỗ trợ giảm cân và tốt cho tim mạch"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Xoài sấy dẻo đặc sản Langfarm Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "92000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Xoài sấy dẻo đặc sản Langfarm Hũ tobita 200g Bịch mawashi 650g"
      },
      {
        "from": "gpt",
        "value": "259000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Xoài sấy dẻo đặc sản Langfarm Hũ tobita 200g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "93000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Xoài sấy dẻo đặc sản Langfarm Bịch mawashi 650g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "92000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Xoài sấy dẻo đặc sản Langfarm Bịch mawashi 650g"
      },
      {
        "from": "gpt",
        "value": "259000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Xoài sấy dẻo đặc sản Langfarm Bịch mawashi 650g Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "93000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Xoài sấy dẻo đặc sản Langfarm Bịch kraft 225g Hũ tobita 200g"
      },
      {
        "from": "gpt",
        "value": "92000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Xoài sấy dẻo đặc sản Langfarm Bịch kraft 225g Bịch mawashi 650g"
      },
      {
        "from": "gpt",
        "value": "259000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Xoài sấy dẻo đặc sản Langfarm Bịch kraft 225g"
      },
      {
        "from": "gpt",
        "value": "93000 đồng, Ăn vặt nông sản, hương vị tự nhiên, dẻo ngon, ít ngọt"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Xoài sấy giòn đặc sản Langfarm Bịch goods 35g Xoài lát"
      },
      {
        "from": "gpt",
        "value": "43000 đồng, Giòn tan từng miếng, trọn vị thiên nhiên, chỉ có tại Langfarm"
      }
    ]
  },
  {
    "conversations": [
      {
        "from": "human",
        "value": "Yến baby thiên nhiên Yumsea x Langfarm Saka 1 viên"
      },
      {
        "from": "gpt",
        "value": "61000 đồng, 100% yến thiên nhiên nguyên chất, không pha độn"
      }
    ]
  }
]


In [7]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
I am about MEME Coin generator AI Chat. I should support user on suggest user the meme name, token name, and totalSupply at the end. The meme coin will be list on uniswap (uni v3)
</reasoning>
<answer>
Hello, I'm Meme Coin Generator AI Agent, Ask me anything about the meme coin that u want. I can support you on that
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# # uncomment middle messages for 1-shot prompting
# def get_gsm8k_questions(split = "train") -> Dataset:
#     data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
#     data = data.map(lambda x: { # type: ignore
#         'prompt': [
#             {'role': 'system', 'content': SYSTEM_PROMPT},
#             {'role': 'user', 'content': x['question']}
#         ],
#         'answer': extract_hash_answer(x['answer'])
#     }) # type: ignore
#     return data # type: ignore

# def get_custom_questions():
#     data = Dataset.from_list(custom_data)  # Convert list to Hugging Face Dataset
#     data = data.map(lambda x: {
#         'prompt': [
#             {'role': 'system', 'content': SYSTEM_PROMPT},
#             {'role': 'user', 'content': x['question']}
#         ],
#         'answer': x['answer']
#     })
#     return data

custom_data = []
for item in dataSetUnformatted:
    # Each item has a list of 'conversations'
    conv = item.get("conversations", [])

    # We assume exactly 1 human and 1 gpt turn per item
    human_msg = None
    gpt_msg = None

    for turn in conv:
        if turn["from"] == "human":
            human_msg = turn["value"]
        elif turn["from"] == "gpt":
            gpt_msg = turn["value"]

    # Append the extracted messages to custom_data
    if human_msg is not None and gpt_msg is not None:
        custom_data.append({
            "question": human_msg,
            "answer": gpt_msg
        })


def get_custom_questions():
    # Convert list to Hugging Face Dataset
    data = Dataset.from_list(custom_data)

    # Map each record to the desired prompt structure
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': x['answer']
    })
    return data

# 3) Retrieve your final Dataset
dataset = get_custom_questions()


# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [8]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 6, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

torch.distributed process group is initialized, but parallel_mode != ParallelMode.DISTRIBUTED. In order to use Torch DDP, launch your script with `python -m torch.distributed.launch


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [9]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3 | Num Epochs = 84
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 1
\        /    Total batch size = 1 | Total steps = 250
 "-____-"     Number of trainable parameters = 36,929,536


-------------------- Question:
Can you suggest a meme coin name inspired by Elon Musk? 
Answer:

            <reasoning>Elon Musk has been associated with meme culture and cryptocurrencies like Dogecoin. A meme coin inspired by him could reference his name, Tesla, or space themes.</reasoning>
            <answer>ElonDogeCoin, TeslaMoonToken, or MarsMuskCoin.</answer>
             
Response:
<think>
Okay, the user is asking for a meme coin name inspired by Elon Musk. I need to think of something catchy and relevant to his influence.

Musk is known for tech, startup, and social media, so the coin should reflect that niche. Maybe something tech-related or BdG+. I should make it sound exciting and possibly money-focused since meme coins often require takers.

I want it to be catchy but not too obvious. Maybe something with a twist on "Dust" to give it a techy feel. Something like DUSTcoin could be nice. It has a mouthful, but it's catchy and personal.

I should also mention the website lin

Step,Training Loss,reward,reward_std,completion_length,kl
1,0.000000,0.000000,0.000000,200.000000,0.000000
2,0.000000,0.000000,0.000000,200.000000,0.000000
3,0.000000,0.000000,0.000000,199.666672,0.000000
4,0.000000,0.000000,0.000000,192.333344,0.000000
5,0.000000,0.000000,0.000000,200.000000,0.000000
6,0.000000,0.000000,0.000000,200.000000,0.000000
7,0.000000,0.000000,0.000000,200.000000,0.000000
8,0.000000,0.000000,0.000000,200.000000,0.000000
9,0.000000,0.000000,0.000000,200.000000,0.000000
10,0.000000,0.000000,0.000000,200.000000,0.000000


Streaming output truncated to the last 5000 lines.
            <answer>ElonDogeCoin, TeslaMoonToken, or MarsMuskCoin.</answer>
             
Response:
<think>
Alright, the user is asking me to suggest a meme coin name inspired by Elon Musk. I know that Elon Musk is known for things like Twitter, SpaceX, X-Com, etc. But how to combine that with a meme coin name that's more accessible.

First, I should think about what meme coin names commonly start with a capital letter. Also, the name should be catchy and something people might recognize as popular enough. Maybe combining elements of social media and a tech term could work. 

Musk's tweets are popular, so maybe something like Mars Twitter or ElonX? But those seem a bit cryptic. How to make them more playful? Maybe "ElonX tweets" but that still feels a bit forced.

I want something that rings a bell as a meme coin, so maybe "MuskMeme" is too generic. Alternatively, combining two popular names like tweets and meme might work, like "MemeX

TrainOutput(global_step=250, training_loss=6.58916360407602e-08, metrics={'train_runtime': 2955.9852, 'train_samples_per_second': 0.085, 'train_steps_per_second': 0.085, 'total_flos': 0.0, 'train_loss': 6.58916360407602e-08})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.78s/it, est. speed input: 1.64 toks/s, output: 19.94 toks/s]


'Calculating pi to a large number of decimal places is a complex task that requires a computational approach, rather than a simple mathematical formula. Here\'s a way to calculate pi using the Monte Carlo method, which is an approximation method that uses random numbers to estimate the value of pi:\n\n**The Monte Carlo Method**\n\nThe Monte Carlo method is based on the idea of simulating the probability of a random walk across a square and circle. Here\'s the basic idea:\n\n1. Draw a square and a circle on a piece of paper.\n2. Generate random points within the square.\n3. Count the proportion of points that fall within the circle.\n4. The ratio of points within the circle to the total number of points is approximately equal to the ratio of the area of the circle to the area of the square, which is pi.\n\n**Mathematical Formulation**\n\nLet\'s denote the following variables:\n\n*   `N`: the number of random points generated\n*   `n`: the number of points within the circle\n*   `pi_appr

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [ ]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [ ]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "Calculate pi."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:23<00:00, 23.29s/it, est. speed input: 2.62 toks/s, output: 19.41 toks/s]


"<reasoning>\nPi (π) is an irrational number that represents the ratio of a circle's circumference to its diameter. It is approximately equal to 3.14159, but its decimal representation goes on indefinitely without repeating.\n\nTo calculate pi, we can use various mathematical formulas and methods, such as the Leibniz formula, the Gregory-Leibniz series, or the Monte Carlo method. However, these methods are not practical for obtaining a high degree of accuracy.\n\nA more practical approach is to use the Bailey-Borwein-Plouffe (BBP) formula, which is a spigot algorithm that allows us to calculate any digit of pi without having to compute the preceding digits.\n\nAnother method is to use the Chudnovsky algorithm, which is a fast and efficient method for calculating pi to a high degree of accuracy.\n\nFor simplicity, we can use the first few terms of the BBP formula to estimate pi:\nπ = 3 + 1/(4/3 - 1/(4/3 - 1/(4/3 - ...))\n\nLet's use this simplified formula to estimate pi:\n\nπ ≈ 3 + 1/(

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
